<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [10]:
import numpy as np
import gensim
import os
import re
from collections import Counter

# Plotting
import squarify
import matplotlib.pyplot as plt
import seaborn as sns

# NLP Libraries
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer

nlp = spacy.load("en_core_web_lg")

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [11]:
review = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [12]:
print(review.shape)
review.head(2)

(28332, 24)


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."


In [14]:
columns = ['id', 'brand', 'primaryCategories', 'reviews.text', 'reviews.title']

review = review[columns]

In [15]:
review.head(2)

,id,brand,primaryCategories,reviews.text,reviews.title
0,AVpgNzjwLJeJML43Kpxn,Amazonbasics,Health & Beauty,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...
1,AVpgNzjwLJeJML43Kpxn,Amazonbasics,Health & Beauty,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...


In [16]:
STOPWORDS = set(STOPWORDS)

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [19]:
review['tokens_rev'] = review['reviews.text'].apply(tokenize)
review['tokens_cat'] = review['primaryCategories'].apply(tokenize)

In [22]:
review['tokens_cat'].head()

0    [health, beauty]
1    [health, beauty]
2    [health, beauty]
3    [health, beauty]
4    [health, beauty]
Name: tokens_cat, dtype: object

In [24]:
id2word = corpora.Dictionary((review['tokens_rev']))

In [26]:
corpus = [id2word.doc2bow(text) for text in review['tokens_rev']]

In [27]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [28]:
lda.print_topics()

[(0,
  '0.179*"good" + 0.049*"batteries" + 0.033*"price" + 0.031*"value" + 0.020*"far" + 0.019*"product" + 0.016*"quality" + 0.016*"battery" + 0.016*"buy" + 0.016*"recommend"'),
 (1,
  '0.034*"case" + 0.027*"kids" + 0.027*"friendly" + 0.027*"child" + 0.022*"loves" + 0.021*"set" + 0.020*"tablet" + 0.018*"kid" + 0.016*"user" + 0.015*"love"'),
 (2,
  '0.040*"tablet" + 0.028*"amazon" + 0.019*"apps" + 0.016*"play" + 0.014*"games" + 0.014*"google" + 0.013*"store" + 0.012*"kindle" + 0.011*"books" + 0.011*"use"'),
 (3,
  '0.035*"size" + 0.032*"screen" + 0.024*"use" + 0.023*"love" + 0.022*"nice" + 0.021*"like" + 0.020*"easy" + 0.020*"good" + 0.014*"light" + 0.014*"tablet"'),
 (4,
  '0.060*"best" + 0.057*"buy" + 0.020*"bought" + 0.015*"dropped" + 0.015*"product" + 0.014*"good" + 0.012*"far" + 0.011*"haven" + 0.011*"working" + 0.011*"thank"'),
 (5,
  '0.046*"battery" + 0.033*"life" + 0.027*"kindle" + 0.017*"screen" + 0.017*"great" + 0.016*"light" + 0.015*"easy" + 0.013*"reader" + 0.011*"use" + 0.

In [29]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [30]:
topics = [' '.join(t[0:5]) for t in words]

In [33]:
for t in topics:
    print(t, '\n')

good batteries price value far 

case kids friendly child loves 

tablet amazon apps play games 

size screen use love nice 

best buy bought dropped product 

battery life kindle screen great 

batteries great price long brand 

amazon batteries long box like 

old year loves tablet bought 

books read love tablet bought 

price apps tablet great kids 

screen charge problem charging kindle 

kindle great bought love gift 

work alexa tap fine echo 

great tablet price use kids 



In [34]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [35]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.227083 -0.098007       1        1  16.626432
2     -0.107496  0.042488       2        1  10.301415
8     -0.229509 -0.013746       3        1  10.147890
14    -0.079255 -0.215214       4        1   7.866875
12    -0.116779  0.005466       5        1   6.419446
3     -0.063247 -0.005057       6        1   6.412938
0      0.238740 -0.132766       7        1   5.419056
10    -0.047050 -0.112157       8        1   5.368189
7      0.185004  0.114583       9        1   5.334826
5      0.000178  0.058539      10        1   4.760062
13     0.148727  0.100310      11        1   4.689107
1     -0.173428 -0.038701      12        1   4.542655
9     -0.145638  0.106121      13        1   4.308856
11     0.028136  0.181828      14        1   4.061895
4      0.134533  0.006313      15        1   3.740356, topic_info=     Category         Freq       Term        Total  loglift  logprob
19    Default  7107.000000  batteries  7107.000000  30.0000  30.0000
35    Default  5712.000000       good  5712.000000  29.0000  29.0000
22    Default  8672.000000      great  8672.000000  28.0000  28.0000
4197  Default  6813.000000     tablet  6813.000000  27.0000  27.0000
18    Default  4802.000000      price  4802.000000  26.0000  26.0000
3157  Default  2928.000000     kindle  2928.000000  25.0000  25.0000
605   Default  2082.000000        old  2082.000000  24.0000  24.0000
10    Default  2243.000000       work  2243.000000  23.0000  23.0000
108   Default  1730.000000       year  1730.000000  22.0000  22.0000
2616  Default  2152.000000      loves  2152.000000  21.0000  21.0000
3173  Default  1501.000000      books  1501.000000  20.0000  20.0000
30    Default  2272.000000        buy  2272.000000  19.0000  19.0000
3     Default  2069.000000    battery  2069.000000  18.0000  18.0000
235   Default  1468.000000       best  1468.000000  17.0000  17.0000
202   Default  2517.000000       kids  2517.000000  16.0000  16.0000
2050  Default  1816.000000       apps  1816.000000  15.0000  15.0000
772   Default  1695.000000      games  1695.000000  14.0000  14.0000
2245  Default  1661.000000     screen  1661.000000  13.0000  13.0000
93    Default  3422.000000       love  3422.000000  12.0000  12.0000
935   Default  1219.000000       read  1219.000000  11.0000  11.0000
134   Default   950.000000       size   950.000000  10.0000  10.0000
24    Default  2414.000000       long  2414.000000   9.0000   9.0000
446   Default  1014.000000      value  1014.000000   8.0000   8.0000
21    Default  1223.000000      brand  1223.000000   7.0000   7.0000
902   Default  2945.000000       easy  2945.000000   6.0000   6.0000
61    Default  1001.000000       life  1001.000000   5.0000   5.0000
174   Default  1915.000000    product  1915.000000   4.0000   4.0000
29    Default  3103.000000     bought  3103.000000   3.0000   3.0000
305   Default   838.000000       case   838.000000   2.0000   2.0000
76    Default   853.000000     brands   853.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
2007  Topic15    57.692944    purpose   116.541550   2.5829  -5.3324
152   Topic15   135.033615    working   376.539276   2.2605  -4.4820
158   Topic15   101.818542    reviews   267.100037   2.3216  -4.7643
42    Topic15    24.471506        ive    38.878101   2.8231  -6.1900
4     Topic15   123.415977       item   378.073914   2.1665  -4.5720
176   Topic15   124.211823      times   402.372070   2.1106  -4.5655
114   Topic15    64.552269    exactly   158.953674   2.3849  -5.2201
752   Topic15    84.611992   problems   288.772888   2.0584  -4.9495
559   Topic15    35.578224       held    77.676407   2.5052  -5.8158
159   Topic15    65.869598       said   229.112595   2.0395  -5.1999
749   Topic15    72.035454       sale   278.476898   1.9338  -5.1104
205   Topic15   148.766525        far  1210.802734   1.1893  -4.3852
180

In [36]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, path, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        stream = doc_stream(path)
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=id2word, workers=4)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=stream, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [43]:
import warnings
warnings.filterwarnings('ignore')

model_list, coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus, 
                                                        path=review['tokens_rev'], 
                                                        start=2, 
                                                        limit=40, 
                                                        step=6)

TypeError: listdir: path should be string, bytes, os.PathLike, integer or None, not Series

In [40]:
# Show graph

limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

NameError: name 'coherence_values' is not defined

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling